# 🎵 DEMO: Phát Hiện Ngáy từ Audio File

Script đơn giản để test model với file audio

## 📝 Hướng dẫn sử dụng:
1. **Thay đổi đường dẫn file audio** ở cell bên dưới (dòng `AUDIO_FILE`)
2. **Chạy tất cả các cell** (Cell → Run All)
3. **Xem kết quả** ở cuối notebook

## ⚙️ Yêu cầu:
- File model: `Model_LeNet5_PhatHienNgay_100x64.h5` phải có trong cùng thư mục
- File audio: WAV, MP3, M4A, FLAC, OGG, AAC


In [1]:
# ============================================
# ⚙️ CẤU HÌNH - CHỈ CẦN THAY ĐƯỜNG DẪN FILE Ở ĐÂY
# ============================================

# 👇 THAY ĐƯỜNG DẪN FILE AUDIO CỦA BẠN VÀO ĐÂY 👇
AUDIO_FILE = "/Users/thanhhuongtran/Documents/lab-proj/mentor/D2T/1_16.wav"  # Ví dụ: "audio/test.wav" hoặc "/Users/name/audio.mp3"

MODEL_FILE = "/Users/thanhhuongtran/Documents/lab-proj/mentor/D2T/Model_Custom_PhatHienNgay_100x64.h5"

# Tên các lớp - QUAN TRỌNG: Phải khớp với label mapping trong training!
# Trong training: Others=0, Normal Snore=1, OSA Snore=2
# Sau khi to_categorical(3): Index 0=Others, Index 1=Normal Snore, Index 2=OSA Snore
CLASS_NAMES = ['Others', 'Normal Snore', 'OSA Snore']  # ⚠️ Đã sửa lại để khớp với training!

print("✅ Đã cấu hình!")
print(f"📂 File audio: {AUDIO_FILE}")
print(f"🤖 File model: {MODEL_FILE}")


✅ Đã cấu hình!
📂 File audio: /Users/thanhhuongtran/Documents/lab-proj/mentor/D2T/1_16.wav
🤖 File model: /Users/thanhhuongtran/Documents/lab-proj/mentor/D2T/Model_Custom_PhatHienNgay_100x64.h5


In [2]:
!pip install pywavelets

In [3]:
# Import các thư viện cần thiết
import numpy as np
import librosa
import pywt
from scipy import ndimage
from tensorflow.keras.models import load_model
from sklearn.decomposition import PCA
import os

print("✅ Đã import tất cả thư viện!")


✅ Đã import tất cả thư viện!


In [4]:
# Kiểm tra file có tồn tại không
if not os.path.exists(AUDIO_FILE):
    print(f"❌ LỖI: Không tìm thấy file audio!")
    print(f"   Đường dẫn: {AUDIO_FILE}")
    print(f"\n💡 Hãy thay đổi AUDIO_FILE ở cell trên!")
else:
    print(f"✅ Đã tìm thấy file audio: {AUDIO_FILE}")

if not os.path.exists(MODEL_FILE):
    print(f"❌ LỖI: Không tìm thấy file model!")
    print(f"   Đường dẫn: {MODEL_FILE}")
else:
    print(f"✅ Đã tìm thấy file model: {MODEL_FILE}")


✅ Đã tìm thấy file audio: /Users/thanhhuongtran/Documents/lab-proj/mentor/D2T/1_16.wav
✅ Đã tìm thấy file model: /Users/thanhhuongtran/Documents/lab-proj/mentor/D2T/Model_Custom_PhatHienNgay_100x64.h5


In [5]:
# Load model
print(f"📦 Đang load model từ: {MODEL_FILE}...")
model = load_model(MODEL_FILE)
print("✅ Đã load model thành công!")


📦 Đang load model từ: /Users/thanhhuongtran/Documents/lab-proj/mentor/D2T/Model_Custom_PhatHienNgay_100x64.h5...
✅ Đã load model thành công!


In [6]:
# Hàm xử lý audio giống hệt training data (theo ảnh)
# Training code: coeff, freqs = pywt.cwt(data, scales, 'morl')
#                pca = PCA(n_components=64)
#                coeff = pca.fit_transform(coeff)
#                np.save(output_file, coeff)
def process_audio_segment(audio_segment, sampling_rate=8000):
    """
    Xử lý một đoạn audio 1 giây giống hệt training data
    - Downsample về 8kHz
    - CWT với Morlet wavelet, scales 1-100
    - PCA giảm về 64 components
    - Output: (100, 64) hoặc shape tương tự file .npy
    """
    # Đảm bảo audio đúng độ dài 1 giây (8000 samples ở 8kHz)
    target_samples = sampling_rate * 1  # 1 giây
    if len(audio_segment) != target_samples:
        if len(audio_segment) < target_samples:
            # Pad nếu ngắn hơn
            audio_segment = np.pad(audio_segment, (0, target_samples - len(audio_segment)), mode='constant')
        else:
            # Cắt nếu dài hơn
            audio_segment = audio_segment[:target_samples]
    
    # Continuous Wavelet Transform (CWT) với Morlet wavelet
    # Scales từ 1 đến 100 (giống training)
    scales = np.arange(1, 101)
    coeff, freqs = pywt.cwt(audio_segment, scales, 'morl')
    # coeff shape: (100, time_samples) - 100 scales, time_samples thời gian
    
    # Áp dụng PCA - GIỐNG HỆT TRAINING CODE (theo ảnh)
    # Training code: pca.fit_transform(coeff) với coeff shape (100, time_samples)
    # PCA.fit_transform(coeff) sẽ:
    #   - Coi mỗi column (time step) là một sample với 100 features (scales)
    #   - Giảm từ 100 features xuống 64 components
    #   - Kết quả: (64, time_samples)
    pca = PCA(n_components=64)
    coeff_pca = pca.fit_transform(coeff)  # Shape: (64, time_samples)
    
    # Transpose để có (time_samples, 64) - giống như khi load từ .npy
    # Training data file .npy có shape (100, 64) sau khi được xử lý
    coeff_pca = coeff_pca.T  # Shape: (time_samples, 64)
    
    # Resize về (100, 64) - giống training data
    # Training data file .npy có shape (100, 64)
    # Cần resize time_samples về 100, giữ nguyên 64 components
    if coeff_pca.shape != (100, 64):
        # Resize cả 2 chiều về đúng (100, 64)
        zoom_0 = 100 / coeff_pca.shape[0]
        zoom_1 = 64 / coeff_pca.shape[1]
        coeff_resized = ndimage.zoom(coeff_pca, 
                                     (zoom_0, zoom_1),
                                     order=1)
    else:
        coeff_resized = coeff_pca
    
    # Đảm bảo shape cuối cùng chính xác là (100, 64)
    if coeff_resized.shape != (100, 64):
        # Force resize nếu vẫn không đúng
        coeff_resized = ndimage.zoom(coeff_resized, 
                                     (100 / coeff_resized.shape[0], 64 / coeff_resized.shape[1]),
                                     order=1)
    
    # Kết quả: (100, 64) - giống file .npy trong training
    return coeff_resized.astype(np.float32)

def process_audio_file(audio_path, segment_length=1.0, sampling_rate=8000):
    """
    Xử lý file audio giống hệt training data
    - Load raw audio (.wav, .mp3, ...)
    - Downsample về 8kHz
    - Chia thành các đoạn 1 giây (giống training: 1000ms)
    - Xử lý từng đoạn: CWT -> PCA -> (100, 64)
    """
    print(f"📂 Đang đọc file audio: {audio_path}")
    
    # Load audio file và downsample về 8kHz (giống training)
    y, sr = librosa.load(audio_path, sr=sampling_rate, mono=True)
    duration = len(y) / sr
    print(f"   ✅ Đã load audio: {duration:.2f} giây, sample rate: {sr} Hz (đã downsample về 8kHz)")
    
    # Chia audio thành các đoạn 1 giây (giống training: duration_ms=1000)
    segment_samples = int(segment_length * sampling_rate)  # 8000 samples = 1 giây
    segments = []
    
    num_segments = int(len(y) / segment_samples)
    if len(y) % segment_samples > 0:
        num_segments += 1
    
    print(f"   🔄 Chia audio thành {num_segments} đoạn (mỗi đoạn {segment_length}s = {segment_samples} samples)")
    
    for i in range(num_segments):
        start = i * segment_samples
        end = start + segment_samples
        
        if end > len(y):
            # Đoạn cuối: pad nếu cần
            segment = y[start:]
            if len(segment) < segment_samples:
                segment = np.pad(segment, (0, segment_samples - len(segment)), mode='constant')
        else:
            segment = y[start:end]
        
        # Xử lý đoạn audio: CWT -> PCA -> (100, 64)
        spectrogram = process_audio_segment(segment, sampling_rate=sampling_rate)
        segments.append(spectrogram)
    
    print(f"   ✅ Đã tạo {len(segments)} spectrograms với shape {segments[0].shape}")
    return segments

print("✅ Đã định nghĩa hàm xử lý audio giống hệt training data!")


✅ Đã định nghĩa hàm xử lý audio giống hệt training data!


In [7]:
# Xử lý audio file giống hệt training data
print("="*50)
print("🔍 ĐANG PHÂN TÍCH AUDIO...")
print("="*50)
print("📝 Quy trình: Load raw audio -> Downsample 8kHz -> Chia 1s/đoạn -> CWT -> PCA -> (100,64)")
print("="*50)

# Xử lý audio: chia thành các đoạn 1 giây và xử lý từng đoạn
spectrograms = process_audio_file(AUDIO_FILE, segment_length=1.0, sampling_rate=8000)

# Kiểm tra shape trước khi reshape
print(f"\n🔍 Debug: Shape của spectrograms:")
for i, spec in enumerate(spectrograms[:3]):  # Chỉ in 3 cái đầu
    print(f"   Spectrogram {i}: {spec.shape}")

# Reshape để phù hợp với input của model (100, 64, 1)
spectrograms_reshaped = np.array([spec.reshape(100, 64, 1) for spec in spectrograms])

print(f"\n✅ Đã chuẩn bị {len(spectrograms_reshaped)} đoạn cho model!")
print(f"✅ Shape mỗi đoạn: {spectrograms_reshaped[0].shape}")


🔍 ĐANG PHÂN TÍCH AUDIO...
📝 Quy trình: Load raw audio -> Downsample 8kHz -> Chia 1s/đoạn -> CWT -> PCA -> (100,64)
📂 Đang đọc file audio: /Users/thanhhuongtran/Documents/lab-proj/mentor/D2T/1_16.wav
   ✅ Đã load audio: 1.00 giây, sample rate: 8000 Hz (đã downsample về 8kHz)
   🔄 Chia audio thành 1 đoạn (mỗi đoạn 1.0s = 8000 samples)
   ✅ Đã tạo 1 spectrograms với shape (100, 64)

🔍 Debug: Shape của spectrograms:
   Spectrogram 0: (100, 64)

✅ Đã chuẩn bị 1 đoạn cho model!
✅ Shape mỗi đoạn: (100, 64, 1)


In [8]:
# Chạy model predict cho tất cả các đoạn
print(f"\n🤖 Đang chạy model predict cho {len(spectrograms_reshaped)} đoạn...")
all_predictions = model.predict(spectrograms_reshaped, verbose=0)

# Tính kết quả trung bình từ tất cả các đoạn
avg_predictions = np.mean(all_predictions, axis=0)
class_idx = np.argmax(avg_predictions)
confidence = float(avg_predictions[class_idx])
class_name = CLASS_NAMES[class_idx]

print(f"✅ Đã hoàn tất predict {len(all_predictions)} đoạn!")
print(f"📊 Kết quả được tính từ trung bình của {len(all_predictions)} đoạn")



🤖 Đang chạy model predict cho 1 đoạn...
✅ Đã hoàn tất predict 1 đoạn!
📊 Kết quả được tính từ trung bình của 1 đoạn


In [9]:
# Hiển thị kết quả chi tiết
print("\n" + "="*50)
print("📊 KẾT QUẢ DỰ ĐOÁN (Trung bình từ tất cả các đoạn)")
print("="*50)
print(f"\n🎯 Lớp dự đoán: {class_name}")
print(f"📈 Độ tin cậy: {confidence*100:.2f}%")
print(f"📦 Số đoạn đã phân tích: {len(all_predictions)}")

print(f"\n📋 Xác suất trung bình cho tất cả các lớp:")
print("-" * 50)
for i, (name, prob) in enumerate(zip(CLASS_NAMES, avg_predictions)):
    bar_length = int(prob * 30)  # Bar dài 30 ký tự
    bar = "█" * bar_length + "░" * (30 - bar_length)
    marker = "👉" if i == class_idx else "  "
    print(f"{marker} {name:15s}: {prob*100:6.2f}% {bar}")

# Hiển thị kết quả từng đoạn
print(f"\n📊 Kết quả từng đoạn:")
print("-" * 50)
segment_results = []
for idx, pred in enumerate(all_predictions):
    seg_class = CLASS_NAMES[np.argmax(pred)]
    seg_conf = float(pred[np.argmax(pred)])
    segment_results.append(seg_class)
    if idx < 5:  # Chỉ hiển thị 5 đoạn đầu
        print(f"   Đoạn {idx+1}: {seg_class} ({seg_conf*100:.1f}%)")

if len(all_predictions) > 5:
    print(f"   ... và {len(all_predictions)-5} đoạn khác")

# Đếm số lượng mỗi class
from collections import Counter
class_counts = Counter(segment_results)
print(f"\n📈 Phân bố kết quả từng đoạn:")
for cls, count in class_counts.items():
    print(f"   {cls}: {count}/{len(segment_results)} đoạn ({count/len(segment_results)*100:.1f}%)")

print("="*50)



📊 KẾT QUẢ DỰ ĐOÁN (Trung bình từ tất cả các đoạn)

🎯 Lớp dự đoán: Others
📈 Độ tin cậy: 99.79%
📦 Số đoạn đã phân tích: 1

📋 Xác suất trung bình cho tất cả các lớp:
--------------------------------------------------
👉 Others         :  99.79% █████████████████████████████░
   Normal Snore   :   0.03% ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
   OSA Snore      :   0.17% ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░

📊 Kết quả từng đoạn:
--------------------------------------------------
   Đoạn 1: Others (99.8%)

📈 Phân bố kết quả từng đoạn:
   Others: 1/1 đoạn (100.0%)


In [10]:
# Hiển thị kết quả dạng dictionary (để dễ sử dụng)
results = {
    'predicted_class': class_name,
    'confidence': round(confidence * 100, 2),
    'num_segments': len(all_predictions),
    'all_probabilities': {
        CLASS_NAMES[0]: round(float(avg_predictions[0]) * 100, 2),
        CLASS_NAMES[1]: round(float(avg_predictions[1]) * 100, 2),
        CLASS_NAMES[2]: round(float(avg_predictions[2]) * 100, 2)
    },
    'segment_distribution': dict(class_counts)
}

print("\n📦 Kết quả dạng dictionary:")
results



📦 Kết quả dạng dictionary:


{'predicted_class': 'Others',
 'confidence': 99.79,
 'num_segments': 1,
 'all_probabilities': {'Others': 99.79,
  'Normal Snore': 0.03,
  'OSA Snore': 0.17},
 'segment_distribution': {'Others': 1}}